In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 300)
pd.set_option("max_colwidth", 200)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn import preprocessing

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
base = pd.read_csv('../Data/BaseData.csv')

In [3]:
base_ordinal = pd.read_csv('../Data/BaseOrdinal.csv')

In [8]:
def Performance_Full_Data(data):
    
    X = data.drop(['SalePrice'], axis=1).values
    Y = data.SalePrice.values
    #Y = np.log(data.SalePrice.values)
    
    MLR = LinearRegression()
    MLR.fit(X,Y)
    R2_MLR = MLR.score(X,Y)
    
    r = Ridge()
    r.fit(X,Y)
    Ridge_Score = r.score(X,Y)
    
    scaler = preprocessing.StandardScaler().fit(X)
    X_scaled = scaler.transform(X)
    alphas = 10**np.linspace(10,-2,100)*0.5
    
    ridgecv= RidgeCV(alphas=alphas, scoring='neg_mean_squared_error')
    ridgecv.fit(X_scaled, Y)
    ridge_best_alpha = Ridge(alpha=round(ridgecv.alpha_,2))
    ridge_best_alpha.fit(X_scaled, Y)
    Ridge_A_Score = ridge_best_alpha.score(X_scaled, Y)
    
    l = Lasso()
    l.fit(X,Y)
    Lasso_Score = l.score(X,Y)
    
    lassocv = LassoCV(alphas=alphas, cv=10, max_iter = 100000)
    lassocv.fit(X_scaled, Y)
    lasso_best_alpha = Lasso(alpha=round(lassocv.alpha_,2))
    lasso_best_alpha.fit(X_scaled, Y)
    Lasso_A_Score = lasso_best_alpha.score(X_scaled, Y)
    
    elastic = ElasticNet(alpha=1, l1_ratio=0.5)
    elastic.fit(X_scaled,Y)
    Elastic_Score = elastic.score(X_scaled, Y)
    
    RF_No_Bootstrap = RandomForestRegressor(n_estimators=100, criterion='squared_error', 
                                            bootstrap=False)
    RF_No_Bootstrap.fit(X_scaled, Y)
    RF_No_Bootstrap_Score = RF_No_Bootstrap.score(X_scaled, Y)
    
    RF_W_Bootstrap = RandomForestRegressor(n_estimators=100, criterion='squared_error', 
                                            bootstrap=True)
    RF_W_Bootstrap.fit(X_scaled, Y)
    RF_W_Bootstrap_Score = RF_W_Bootstrap.score(X_scaled, Y)
    
    gbm = GradientBoostingRegressor()
    gbm.fit(X_scaled,Y)
    GBM_Score = gbm.score(X_scaled, Y)
    
    xgb = XGBRegressor()
    xgb.fit(X_scaled, Y)
    XGB_Score = xgb.score(X_scaled, Y)
    
    #elas_grid = dict()
    #elas_grid['alpha'] = arange(0.1,1,0.1)
    #elas_grid['l1ratio'] = arange(0.1,1,0.1)
    
    #elasticgrid = GridSearchCV(ElasticNet(), elas_grid, 
    #                           scoring='neg_mean_absolute_error', n_jobs=-1)
    
    #elasticgrid.fit(X_scaled, Y)
    
    #results = search.fit(X_scaled, Y)
    
    #elasticgrid = ElasticNet(alpha=results.best_params['alpha'],
    #                        l1_ratio=results.best_params_['l1_ratio'])
    #elasticgrid.fit(X_scaled, Y)
    #Elastic_Score = elasticgrid.score(X_scaled, Y)
    
    d = {'Model': ["Multiple Linear Regression", r, ridge_best_alpha, 
                   l, lasso_best_alpha, elastic, "RF No Bootstrap", "RF With Bootstrap",
                   "Gradient Boosting", "Xtreme Gradient Boosting"],
         'Performance (R2)': [R2_MLR, Ridge_Score, Ridge_A_Score, 
                                Lasso_Score, Lasso_A_Score, Elastic_Score,
                                RF_No_Bootstrap_Score, RF_W_Bootstrap_Score,
                                GBM_Score, XGB_Score]}
    
    results = pd.DataFrame(data=d)
    return results



In [9]:
Performance_Full_Data(base)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=6.67842e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.464e+11, tolerance: 1.452e+09
  model = cd_fast.enet_coordinate_descent(


,Model,Performance (R2)
0,Multiple Linear Regression,0.939686
1,Ridge(),0.938051
2,Ridge(alpha=37.82),0.938539
3,Lasso(),0.939604
4,Lasso(alpha=152.69),0.936148
5,ElasticNet(alpha=1),0.911685
6,RF No Bootstrap,1.000000
7,RF With Bootstrap,0.984449
8,Gradient Boosting,0.963440
9,Xtreme Gradient Boosting,0.998428


In [10]:
Performance_Full_Data(base_ordinal)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.08934e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.529e+11, tolerance: 1.452e+09
  model = cd_fast.enet_coordinate_descent(


,Model,Performance (R2)
0,Multiple Linear Regression,0.895422
1,Ridge(),0.895419
2,Ridge(alpha=115.51),0.894834
3,Lasso(),0.895421
4,Lasso(alpha=266.83),0.894394
5,ElasticNet(alpha=1),0.880969
6,RF No Bootstrap,1.000000
7,RF With Bootstrap,0.985259
8,Gradient Boosting,0.963750
9,Xtreme Gradient Boosting,0.998786


In [7]:
##Follow Ups = Elastic optimization with Grid Search, SVR